In [1]:
import pandas as pd 

In [2]:
nt = pd.read_csv('netflix_titles.csv')

In [3]:
import pandas as pd
import string
from wordcloud import WordCloud
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder

In [4]:
nt.drop(['show_id'] , axis=1 , inplace=True) 

In [5]:
nt.director.fillna('Unknown' , inplace=True)
nt.cast.fillna('Unknown' , inplace=True)
nt.country.fillna('Unknown' , inplace=True)
nt.rating.fillna('NR' , inplace=True)
nt.date_added.fillna('January 31, 2021' , inplace=True)

In [6]:
date_added_day=[]
date_added_month=[]
date_added_year =[]
def data_process(line):
    line = [char for char in line if char not in string.punctuation]
    line = "".join(line)
    line=line.split()
    date_added_day.append(line[1])
    date_added_month.append(line[0])
    date_added_year.append(line[2])
nt.date_added.apply(data_process)
nt['date_added_day'] = date_added_day
nt['date_added_month'] = date_added_month
nt['date_added_year'] = date_added_year
nt.drop(['date_added'] , axis=1 , inplace=True)


In [7]:
  nt.date_added_month.replace({
    'January':1,
    'February':2,
    'March':3,
    'April':4,
    'May':5,
    'June':6,
    'July':7,
    'August':8,
    'September':9,
    'October':10,
    'November':11,
    'December':12,
} , inplace=True)
le = LabelEncoder()
nt.rating = le.fit_transform(nt.rating)





In [8]:
nt.cast_crew = nt.cast + ","+nt.director
    

<ipython-input-8-91c1ee823287>:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  nt.cast_crew = nt.cast + ","+nt.director


In [9]:
nt.cast_crew[0]

'João Miguel, Bianca Comparato, Michel Gomes, Rodolfo Valente, Vaneza Oliveira, Rafael Lozano, Viviane Porto, Mel Fronckowiak, Sergio Mamberti, Zezé Motta, Celso Frateschi,Unknown'

In [10]:

def text_process_cast_crew_col(mess):
    mess = mess.replace(" ", "")
    mess = mess.replace("&" , ",")
    mess = mess.replace("-","_")
    mess = mess.replace("." , "_")
    #mess = mess.split(",")
    return mess
def text_process_country_col(mess):
    mess = mess.replace(" ", "")
    mess = mess.replace("&" , ",")
    #mess = mess.split(",")
    return mess

In [11]:
nt.cast_crew = nt.cast_crew.apply(text_process_cast_crew_col)
nt.country = nt.country.apply(text_process_country_col)

In [12]:
nt.head()

,type,title,director,cast,country,release_year,rating,duration,listed_in,description,date_added_day,date_added_month,date_added_year
0,TV Show,3%,Unknown,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,2020,8,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...,14,8,2020
1,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,2016,8,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...,23,12,2016
2,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,2011,5,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow...",20,12,2018
3,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",UnitedStates,2009,4,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi...",16,11,2017
4,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",UnitedStates,2008,4,123 min,Dramas,A brilliant group of students become card-coun...,1,1,2020


In [13]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sreeraj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Binarising Listed In

In [14]:
def text_processor_listed_in(mess):
    non_punc = [char for char in mess if char not in  string.punctuation]
    non_punc = "".join(non_punc)
    return[word for word in non_punc.split() if word not in stopwords.words("english")]
bow_transformer_listed_in = CountVectorizer(analyzer=text_processor_listed_in).fit(nt['listed_in'])
binary_listed_in = bow_transformer_listed_in.transform(nt.listed_in)

## Binarising Description

In [87]:
def text_process_description(mess):
    non_punc = [char for char in mess if char not in  string.punctuation]
    non_punc = "".join(non_punc)
    return[word for word in non_punc.split() if word not in stopwords.words("english")]
bow_transformer_description = CountVectorizer(analyzer=text_process_description).fit(nt['description'])
binary_discription = bow_transformer_description.transform(nt.description)

## Binarising Cast_Crew 

In [88]:
def text_process_cast_crew(mess):
    mess = mess.replace(" ", "")
    mess = mess.replace("&" , "")
    mess = mess.replace("-","_")
    mess = mess.replace("." , "_")
    mess = mess.split(",")
    return mess
bow_tranformer_cast = CountVectorizer(analyzer=text_process_cast).fit(nt['cast_crew'])
binary_cast = bow_tranformer_cast.transform(nt.cast_crew)

## Binarising Country 

In [16]:
def text_process_country(mess):
    return mess.split(",")
bow_transformer_country =CountVectorizer(analyzer=text_process_country).fit(nt['country'])
binary_country = bow_transformer_country.transform(nt.country)


# Making Sparse Matrix 

In [89]:
df1=pd.DataFrame.sparse.from_spmatrix(binary_cast)
df1.columns=bow_tranformer_cast.get_feature_names()


In [90]:
df2= pd.DataFrame.sparse.from_spmatrix(binary_listed_in)
df2.columns= bow_transformer_listed_in.get_feature_names()


In [91]:
df3= pd.DataFrame.sparse.from_spmatrix(binary_discription)
df3.columns= bow_transformer_description.get_feature_names()


In [92]:
df4= pd.DataFrame.sparse.from_spmatrix(binary_country)
df4.columns= bow_transformer_country.get_feature_names()


In [93]:
mat = pd.concat([df1 , df2  , df4, nt.date_added_year , nt.release_year , nt.title] , axis=1)

In [94]:
from sklearn.cluster import KMeans

In [98]:
data = input("enter search: ")
data= data.replace(" " , "")
if data in mat.columns:
    print(mat.title[mat[data]==1])
else:
    print("not found ")

enter search: Aamir Khan
100                        3 Idiots
507                 Andaz Apna Apna
1595                         Dangal
1739    Dhobi Ghat (Mumbai Diaries)
1756                            Dil
1757                 Dil Chahta Hai
1758                Dil Dhadakne Do
3521                         Lagaan
3848          Madness in the Desert
3912                           Mann
4872                             PK
5077                Raja Hindustani
5097                Rang De Basanti
5464               Secret Superstar
5968                        Talaash
Name: title, dtype: object


In [25]:
mat1 = pd.concat([df , df1 , df2  , df4, nt.date_added_year , nt.release_year  , nt.rating ] , axis=1)

In [26]:
sns_kmean = KMeans(n_clusters=10)
sns_kmean.fit(mat1)

C:\Users\Sreeraj\anaconda3\lib\site-packages\sklearn\utils\validation.py:508: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


KMeans(n_clusters=10)

In [27]:
nt['labels'] = sns_kmean.labels_

In [28]:
import numpy as np

In [36]:
data = input("enter search: ")
if data in list(nt.title):
  label = nt.labels[nt.title == data]
  label = np.array(label)
  print(nt.title[nt.labels == label[0]])
else:
  print("not found")

enter search: Zero
8                            706
14                          3022
17                        22-Jul
18                        15-Aug
20          ​​Kuch Bheege Alfaaz
                  ...           
7735              Young & Hungry
7747                      اشتباك
7764                        Zero
7768    Zindagi Kitni Haseen Hay
7783                      Zubaan
Name: title, Length: 1367, dtype: object
